# Requirements

In [ ]:
# RUN THIS CELL FIRST!
!pip install -q langchain pypdf2 pandas matplotlib tiktoken textract openai faiss-cpu streamlit huggingface_hub pypdfium2 InstructorEmbedding sentence-transformers pypdf googletrans==3.1.0a0 python-docx contractions gradio -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 35.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.1/8.1 MB 72.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 24.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 70.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 7.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 271.0/271.0 kB 26.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 87.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done

# Importing Libraries

In [ ]:
# Importing Libraries
from langchain import LLMChain
from langchain.docstore.document import Document
from langchain.document_loaders import PyPDFLoader
from langchain.memory import ConversationBufferMemory
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import VectorStore
from langchain.vectorstores.faiss import FAISS
from langchain.embeddings import HuggingFaceInstructEmbeddings
from InstructorEmbedding import INSTRUCTOR
from langchain.chat_models import ChatOpenAI
from langchain.llms import HuggingFaceHub
from langchain.document_loaders import Docx2txtLoader
from langchain.document_loaders import GoogleDriveLoader
from langchain.prompts import PromptTemplate
from langchain.document_loaders import TextLoader
from langchain import OpenAI
from pypdf import PdfReader
import os
from bs4 import BeautifulSoup
import contractions
import pypdfium2 as pdfium
import pandas as pd
import re
import glob
import unicodedata
from typing import Any, Dict, List
from googletrans import Translator
import docx
import gradio as gr
import time


/usr/local/lib/python3.10/dist-packages/InstructorEmbedding/instructor.py:7: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import trange


## Download documents to the file section of the notebook

In [ ]:
folder_id = '1O4AO1HxRGbL-IfG__UpJORo50Hl0oN9z'

In [ ]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

folders = [folder_id]

for folder in folders:
  file_list = drive.ListFile({'q': f"'{folder}' in parents and trashed=false"}).GetList()
  for file in file_list:
    file_id = file['id']
    file_name = file['title']

    downloaded = drive.CreateFile({'id': file_id})
    downloaded.FetchMetadata(fetch_all=True)
    downloaded.GetContentFile(downloaded.metadata['title'])


# Helper functions

In [ ]:
#translate answer to English if the question is brought in another language
translator = Translator()
def detect_and_translate(text,answer):
    result_lang = translator.detect(text)
    print(result_lang.lang)
    if result_lang.lang == "en":
        translate_text = answer
    else:
        translate_text = translator.translate(answer,src='auto', dest=result_lang.lang)
    return translate_text

#translate the text to English
def detect_and_translate_ques(text):
    result_lang = translator.detect(text)
    print(result_lang.lang)
    if result_lang.lang == "en":
        translate_text = text
    else:
        translate_text = translator.translate(text,src='auto', dest="en")
    return translate_text


#convert text to document objects
def text_to_docs(text: str) -> List[Document]:
    if isinstance(text, str):
        # Take a single string as one page
        text = [text]
    page_docs = [Document(page_content=page) for page in text]
    # Add page numbers as metadata
    for i, doc in enumerate(page_docs):
        doc.metadata["page"] = i + 1

    return doc

#clean text data
def text_clean(text: str) -> str:
    text = re.sub(r"(\w+)-\n(\w+)", r"\1\2",text)
    text=contractions.fix(text)
    # Fix newlines in the middle of sentences
    text = re.sub(r"(?<!\n\s)\n(?!\s\n)", " ", text.strip())
    text = re.sub(r"(?<!\r\s)\r(?!\s\r)", " ", text.strip())
    # Remove multiple newlines
    text = re.sub(r"\n\s*\n", "\n\n", text)
    text = re.sub(r'https?://\S+', '', text)
    # Remove Unicode data
    text=unicodedata.normalize('NFKD', text)
    return text


#convert pdf to text
def pdfium_get_text(data: bytes) -> str:
    output = ""
    pdf = pdfium.PdfDocument(data)
    for i in range(len(pdf)):
        page = pdf.get_page(i)
        textpage = page.get_textpage()
        text = textpage.get_text_range()
        text = text_clean(text)
        output+=text + "\n"
    return output

#save .docxs to a dictionary
def loadDocumentsFromLocal(file_paths):
    file_contents_docx = {}
    for file_path in file_paths:
        try:
          if ".docx" in file_path:
            doc = docx.Document(file_path)
            doc_content = '\n'.join([paragraph.text for paragraph in doc.paragraphs])
            file_name = file_path.split('/')[-1][:-5]
            file_contents_docx[file_name] = doc_content
        except:
          pass
    return file_contents_docx



# Processing all pdf files

In [ ]:
docs=[]
pdf_files =glob.glob('/content/*.pdf')
for pdf in pdf_files:
  with open(pdf,"rb") as f:
    data = f.read()
    entire_text=pdfium_get_text(data)
    mydoc=text_to_docs(entire_text)
    docs.append(mydoc)



# Splitting docs into chunks

In [ ]:
##### Splitting docs into chunks #########
splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=80,length_function=len)
chunked_docs = splitter.split_documents(docs)